# This notebook uploads all the files in the af_knowledge_base directory to OpenAI and then creates a vector store and an assistant linked to that store.



# First upload all files to OpenAI and collect their IDs

In [1]:
import os
from openai import OpenAI

# Initialize the OpenAI client
import os
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
# Directory containing the files
directory_path = '/Users/adamhunter/Documents/misc/actualism_chat/scrapy_spider/output_files'

# Function to upload a file to OpenAI and return the file ID
def upload_file(file_path):
    with open(file_path, 'rb') as file:
        response = client.files.create(
            file=file,
            purpose='assistants'
        )
        return response.id  # Access the 'id' attribute directly

file_ids = []
count = 0
for filename in os.listdir(directory_path):
    if count < 5000: # Count for safety, since you are charged by storage you dont want this to go wild
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            file_id = upload_file(file_path)
            file_ids.append(file_id)
            print(f"Uploaded {filename} with ID {file_id}")
            count += 1


Uploaded www.actualfreedom.com.au---richard---catalogue---psittacisms.htm.txt with ID file-YVMXDbpabMdSzQ0cZcQImpqR
Uploaded actualfreedom.com.au---richard---listbcorrespondence---listb07.htm.txt with ID file-j10dtasYvwOabjtuqDldgIVE
Uploaded actualfreedom.com.au---actualism---peter---list-c---pc-corr07.htm.txt with ID file-sfpBA1tNkHmbgwcus8wfiU2e
Uploaded actualfreedom.com.au---actualism---peter---selected-correspondence---corr-seth.htm.txt with ID file-hZXSyIccS30fKFFfQAV9M62l
Uploaded actualfreedom.com.au---richard---listbcorrespondence---listb17.htm.txt with ID file-nq3QE8xLdccKdwMWAU3pWgGj
Uploaded actualfreedom.com.au---actualism---peter---list-c---pc-corr17.htm.txt with ID file-TK6lXtuRqg4fuAaVTgC7Q7UF
Uploaded actualfreedom.com.au---richard---listafcorrespondence---listaf44g.htm.txt with ID file-wy5REBIDlACGCCThEUsRxVaZ
Uploaded www.actualfreedom.com.au---actualism---peter---selected-correspondence---corr-time.htm.txt with ID file-9yDGSy3iB64ijgmVRIRt7o07
Uploaded www.actualfr

# Create an empty vector store

In [ ]:
# Create the vector store first without any files
vector_store = client.beta.vector_stores.create(
    name="AF Knowledge Base Vector Store 3"
)
vector_store_id = vector_store.id
print("Vector store created with ID:", vector_store_id)


Vector store created with ID: vs_fTUlICTQMr24XmQ4jnar3Atg


# Connect files to the vector store in batches

In [ ]:
# The maximum number of file IDs allowed per batch
MAX_BATCH_SIZE = 100

# Function to create batches of file IDs
def create_batches(file_ids, batch_size):
    for i in range(0, len(file_ids), batch_size):
        yield file_ids[i:i + batch_size]

# Create and upload batches
for batch in create_batches(file_ids, MAX_BATCH_SIZE):
    vector_store_file_batch = client.beta.vector_stores.file_batches.create(
        vector_store_id=vector_store_id,
        file_ids=batch
    )
    print(f"Batch created with ID: {vector_store_file_batch.id}, Status: {vector_store_file_batch.status}")

print("All batches have been created and are being processed.")

Batch created with ID: vsfb_e157da43ab8e4382bc6a79dbf7c6d9aa, Status: in_progress
Batch created with ID: vsfb_99604a727c86450ea46497ebb9670e87, Status: in_progress
Batch created with ID: vsfb_e9d83a188c674d24a5d157cf1f267a0f, Status: in_progress
Batch created with ID: vsfb_e2eed13012ec40f49e5e1aba7d5ba266, Status: in_progress
Batch created with ID: vsfb_51ca604980c04de4b66a0e4410ac56e6, Status: in_progress
Batch created with ID: vsfb_d4c09c17744947ff94c772bc7f6043fd, Status: in_progress
Batch created with ID: vsfb_294de41655e748d3b62604c5aa826dbd, Status: in_progress
Batch created with ID: vsfb_c7b2278d8f8a45f8aeeb6c2ab2cffa7f, Status: in_progress
Batch created with ID: vsfb_a6b374446485409fb3805dbcaa1238db, Status: in_progress
Batch created with ID: vsfb_01f2d355eace44f0b926e890e568d0d2, Status: in_progress
Batch created with ID: vsfb_9e76b6a7593b4e06b594b245e34184f4, Status: in_progress
Batch created with ID: vsfb_8ac51b0b34604f4eb74f06562ff3ce20, Status: in_progress
Batch created wi

# Create an assistant linked to the vector store

In [4]:
# Create an assistant with the file_search tool linked to the vector store
my_assistant = client.beta.assistants.create(
    model="gpt-3.5-turbo-0125",
    name="AF Knowledge Base Assistant 3",
    instructions="""You answer user queries using your knowledge base of the actual freedom trust website. You are sure to avoid filling in gaps with your own reasoning and stick to what is in the files.""",
    tools=[{"type": "file_search"}],
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store_id]
        }
    }
)

print("Assistant created:")
print(my_assistant)

Assistant created:
Assistant(id='asst_mZAeytPh7TSa4Y6g6QsEtRJd', created_at=1715025807, description=None, instructions='You answer user queries using your knowledge base of the actual freedom trust website. You are sure to avoid filling in gaps with your own reasoning and stick to what is in the files.', metadata={}, model='gpt-3.5-turbo-0125', name='AF Knowledge Base Assistant 3', object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_fTUlICTQMr24XmQ4jnar3Atg'])), top_p=1.0)
